In [1]:
import pydantic
print(pydantic.__version__)
from pydantic import BaseModel, Field, model_validator

2.11.7


In [1]:
import os
import pandas as pd
from llama_cloud_services import LlamaExtract
from pydantic import BaseModel, Field, model_validator
#from mi9 import generate_income_statement_schema 
from realized import generate_realized_schema
from dotenv import load_dotenv


ModuleNotFoundError: No module named 'realized'

In [ ]:
FISCAL_YEAR = 2024  #Change the year if you want different years
IncomeStatement_2024_25 = generate_realized_schema(FISCAL_YEAR)

# PDF_ROOT = "private_universities/university_pdfs"  # Change this to the point to the directory where you are storing the pdfs after scraping
PDF_ROOT = "sample" 
OUTPUT_ROOT = "output_incomestatement_final"  # Make this point to the directory/folder where you want to store the excel files with information extracted
os.makedirs(OUTPUT_ROOT, exist_ok=True)  
# AGENT_ID = "43c63d27-55a6-4268-abb4-26d9ed82480f" #Different based on what agent you are using. Check llamacloud for the list of agents we have under the pro account
# api_key = os.getenv("LLAMACLOUD_API_KEY")
AGENT_ID = "2e5f9f3c-c1ee-4f4f-9ad3-98d2de48e1f5" #enrollment grabs latest data no matter which yr

load_dotenv() #make sure the API key is in the .env file

extractor = LlamaExtract(project_id = '8c10e62e-3810-4193-915d-d2d11105826d')

In [ ]:
# extractor = LlamaExtract(
#     api_key="llx-63CU3PdyDo0d230ureocmy9JOHgnPwYgE2HETi55DqzYCIpy", # Add your Llamacloud API Key 
#     project_id="8c10e62e-3810-4193-915d-d2d11105826d"   #Change the project ID only if Luis has asked you. This is dependent on the llamacloud account
# )


#agent = extractor.create_agent(name = "endowment-parser-2024", data_schema=EndowmentAndInvestmentLevels_2024_25)

agent = extractor.get_agent(id = AGENT_ID)

#uncomment the following lines if you updated the schema
agent.data_schema = IncomeStatement_2024_25.schema()
agent.save()
agent = extractor.get_agent(id = AGENT_ID)


In [ ]:
agent.data_schema

The following two cell blocks extract all schools' info into one excel file per school.

In [ ]:
def process_school(school_name, school_dir):
    combined   = {}
    first_keys = None

    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=[f"{FISCAL_YEAR - 1}-{str(FISCAL_YEAR)[-2:]}"])
        df.index.name = "Metric"
        outfile = os.path.join(OUTPUT_ROOT, f"{school_name}.xlsx")
        df.to_excel(outfile)
        print(f"Saved output to {outfile}")
    else:
        print(f"No PDF data found for {school_name}")

In [ ]:
# Loop over schools
for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue
    print(f"Processing school: {school}")
    process_school(school, school_dir)

print("Extraction complete.")

The following cell block extracts all the schools' info into one excel sheet but in different tabs.

In [ ]:
OUTPUT_FILE = os.path.join(OUTPUT_ROOT, "all_schools.xlsx")

writer = pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl")

for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue

    combined   = {}
    first_keys = None
    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {school}/{fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=[f"{FISCAL_YEAR - 1}-{str(FISCAL_YEAR)[-2:]}"])
        df.index.name = "Metric"
        sheet_name = school[:31]
        df.to_excel(writer, sheet_name=sheet_name)
    else:
        print(f"No data for {school}.")

writer.close()
print(f"All schools written to {OUTPUT_FILE}")

In [ ]:
#Combine all the tabs into one sheet if wanted
file_path   = "output_incomestatement_final/all_schools.xlsx"  #Change this if need be
output_path = "output_incomestatement_final/all_schools_combined.xlsx"  #Change this if need be
raw = pd.read_excel(file_path, sheet_name=None, index_col=0)

school_series = {
    school: df.iloc[:, 0]                      # first (only) value column
    for school, df in raw.items()
}

df_comb = pd.DataFrame(school_series).T
df_comb.index.name = "School"                 
df_comb.insert(0, "Year", f"{FISCAL_YEAR - 1}–{FISCAL_YEAR}")

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    df_comb.to_excel(writer, sheet_name="Combined")

print("Saved:", output_path)